I initialize the list with the function below, based on the algorithm on slide 9 of the pdf. 

In [ ]:
function initialize_list(X::particle)
    dx = zeros(3)
    mc = zeros(Int, 3)
    head = zeros(Int, nCel)
    list = zeros(Int, nPart)
    
    for i in 1:nPart # Loop over particles 
        for j in 1:dim # Compute vector cell index
            dx[j] = X.q[j,i] + L/2
            
            if dx[j] < 0 # Apply PBC
                dx[j] += L
            elseif dx[j] > L
                dx[j] -= L
            end
            
            mc[j] = floor(Int, dx[j]/L*M)
            
            if mc[j] < 0 # Ensure that 0 <= mci <= M-1
                mc[j] = 0
            elseif mc[j] >= M
                mc[j] = M-1
            end
        end
        c = 1 + mc[1] + mc[2]*M + mc[3]*M^2

        list[i] = head[c]
        head[c] = i
    end
    
    return head, list
end

The function below decodes the list and computes the force, based on the algorithm from page 10 of the pdf. 

In [ ]:
function compute_force(X::particle)
    X.f = zeros(dim, nPart)
    clear_list()
    head, list = initialize_list(X)
    mc = zeros(Int, 3)
    dx = zeros(3)
    mci = zeros(Int, 3)

    for i in 1:nPart # (loop 1) Loop over particles 
        for j in 1:dim
            dx[j] = X.q[j,i] + L/2
            
            if dx[j] < 0
                dx[j] += L
            elseif dx[j] > L
                dx[j] -= L
            end
            
            mc[j] = floor(Int, dx[j]/L*M)
            
            if mc[j] < 0
                mc[j] = 0
            elseif mc[j] >= M
                mc[j] = M-1
            end
        end
        
        for mci[1] in (mc[1]-1):(mc[1]+1), # (loop 3) Loop over 27 neighboring cells
            mci[2] in (mc[2]-1):(mc[2]+1),
            mci[3] in (mc[3]-1):(mc[3]+1)

            for j in 1:dim # Ensure that 0 <= mci <= M-1
                if mci[j] < 0
                    mci[j] = 0
                elseif mci[j] >= M
                    mci[j] = M-1
                end
            end
            c = 1 + mci[1] + mci[2]*M + mci[3]*M^2
            j = head[c]

            while j != 0 # Check neighbors
                if i < j # Don't count double pairs
                    dr = X.q[:,i] - X.q[:,j]
                    r = distPBC(dr)
                    ff = LJ_force(r) # returns ff = 0 if r > rc 
                    X.f[:,i] += ff*r
                    X.f[:,j] -= ff*r
                end
                j = list[j]
            end
        end
    end
   
    return X.f
end

Applies PBC on the distance before computing the force:

In [ ]:
function distPBC(dr)
    for i in 1:dim
        if dr[i] > L/2 
            dr[i] -= L
        elseif dr[i] <= -L/2
            dr[i] += L
        end
    end
    
    return dr
end

Applies PBC to remap particles that have left the box:

In [ ]:
function PBC(q) # position between -L/2 and L/2
    for i in nPart
        for j in 1:dim
            if q[j,i] < -L/2 
                q[j,i] += L
            elseif q[j,i] >= L/2 
                q[j,i] -= L
            end
        end
    end
    
    return q
end


Euler-Maruyama to integrate over Nsteps:

In [ ]:
function EM(R::Int, dW::Array{Float64,2}, X::particle)
    h = R*dt
    Nsteps = trunc(Int, N/R)

    for i = 1:Nsteps
        Winc = _addnoise(R, dW, i)
        X.q +=  h*X.p
        X.p +=  h*X.f - γ*X.p*h + sqrt(2*γ/β)*Winc
        
        X.q = PBC(X.q)
        X.f = compute_force(X)
    end
    
    return X.q[1,1]
end